In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df=pd.read_excel("Assignment - ML - Supporting Data.xlsx")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
def important_features(dataset):
    data=dataset.copy()
    for i in range(0,data.shape[0]):
        data['tags'].fillna('', inplace=True)
        data['hashtags'].fillna('', inplace=True)
        data["feature"]=data["tags"]+' '+data["hashtags"]
    return data

In [ ]:
df=important_features(df)
df["feature"]

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
# Importing vectoriser
vec=TfidfVectorizer()

In [ ]:
# vecs=vec.fit_transform(df["feature"].apply(lambda x:np.str_(x)))
tfidf_matrix_train = vec.fit_transform(train_data['feature'])
tfidf_matrix_test = vec.transform(test_data['feature'])

tfidf_matrix_test.shape
tfidf_matrix_train.shape
train_data['interactions']

In [ ]:
sim=cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
sim

In [ ]:
print("sim shape:", sim.shape)
print("interactions shape:", train_data['interactions'].shape)

In [ ]:
user_interactions = (train_data['interactions'] > 0).astype(int).values

# Calculate the interaction rate for each test item
interaction_rates = np.sum(user_interactions * (sim > 0.3), axis=1) / np.sum(sim > 0.3, axis=1)

interaction_rates

In [ ]:
# Predict interactions based on the modified logic
predicted_interactions = (interaction_rates > 0.3).astype(int)
predicted_interactions

In [ ]:
# Compare predicted labels with actual labels
actual_labels = (test_data['interactions'] > 0).astype(int).values
actual_labels

In [ ]:
# Calculate accuracy
accuracy = np.mean(predicted_interactions == actual_labels)
accuracy